# ACS Age Distribution in Phoenix Urban Villages, City of Phoenix, U.S., Maricopa County, Arizona, and Comparable Cities

For population by age 1-year data
- https://www.census.gov/data/developers/data-sets/acs-1year.html
- https://api.census.gov/data/2010/acs/acs1/variables.html
- https://api.census.gov/data/2021/acs/acs1/variables.html

For population by age 5-year data
- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://api.census.gov/data/2013/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

** Note: Census Block Groups (which are aggregated to Phoenix Urban Village areas) are not available until 2013. 

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
import getters as get
import utilcalcs as calc
import geo_agg
from acs_age_vars import *

In [3]:
#Search parameters
y1 = '2021'
y0_5 = '2013' #for urban villages
y0_1 = '2010'

cols = 'group(B01001)'

source5 = 'acs/acs5'
source1 = 'acs/acs1'

var_cols = ['GEO_ID','B01001_001E','B01001_001M']+age_vars

## Population by Age - ACS 1 Year for larger geos
##### <10 years, 10-24, 25-39, 40-54, 55+
Note - does not include Indianapolis, which is not available as a consolidated city in the 1-year data set

In [4]:
def group_age(df,year):
    df.rename(columns={'B01001_001E':'tot_E','B01001_001M':'tot_M'},inplace=True)
    df['p_u10_E'] = df.loc[:,u10_e].sum(axis=1)
    df['p_u10_M'] = df.apply(lambda x: (calc.get_moe(x[u10_m])),axis=1)
    df['p_1024_E'] = df.loc[:,genz_e].sum(axis=1)
    df['p_1024_M'] = df.apply(lambda x: (calc.get_moe(x[genz_m])),axis=1)
    df['p_2539_E'] = df.loc[:,mil_e].sum(axis=1)
    df['p_2539_M'] = df.apply(lambda x: (calc.get_moe(x[mil_m])),axis=1)
    df['p_4054_E'] = df.loc[:,genx_e].sum(axis=1)
    df['p_4054_M'] = df.apply(lambda x: (calc.get_moe(x[genx_m])),axis=1)
    df['p_o55_E'] = df.loc[:,boom_e].sum(axis=1)
    df['p_o55_M'] = df.apply(lambda x: (calc.get_moe(x[boom_m])),axis=1)
    df = df.drop(columns=age_vars)
    
    #add end-year into column name
    for col in df.columns[1:]:
        df.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return df

## City of Phoenix, Maricopa, AZ, U.S., other cities

In [5]:
def year_table(source,year,cols):
    ph = get.get_phx(source,year,cols)
    mar = get.get_maricopa(source,year,cols)
    us = get.get_us(source,year,cols)
    az = get.get_az(source,year,cols)
    cit = get.get_comp_cities(source,year,cols)
    Y = pd.concat([ph,mar,us,az,cit])
    Y = Y[var_cols]
    Y = get.clean_data(Y,Y.columns)
    Y = group_age(Y,year)
    Y = geo_agg.sumgeo_cv(Y,'GEO_ID')
    return Y

In [6]:
Y1 = year_table(source1,y1,cols)
Y0 = year_table(source1,y0_1,cols)

In [7]:
Y0Y1 = pd.merge(Y0,Y1,how='left',on='GEO_ID')

In [8]:
change_vars = ['tot','p_u10','p_1024','p_2539','p_4054','p_o55']

Y0Y1 = calc.calc_change(Y0Y1,y0_1,y1,change_vars)

In [9]:
rename_geos = {'0455000':'Phoenix', '04013':'Maricopa', '0100000US':'US', '0400000US04':'AZ',\
               '0473000':'Tempe','0465000':'Scottsdale', '0427820':'Glendale',\
               '4865000':'San Antonio', '4819000':'Dallas', '1235000':'Jacksonville'}

In [10]:
Y0Y1['GEO_ID'] = Y0Y1.GEO_ID.map(rename_geos)

In [11]:
Y0Y1

,GEO_ID,p_2539_10E,p_2539_10M,p_2539_10C,p_4054_10E,p_4054_10M,p_4054_10C,p_o55_10E,p_o55_10M,p_o55_10C,...,p_4054_1021E,p_4054_1021M,p_4054_1021C,p_4054_10P,p_4054_21P,p_o55_1021E,p_o55_1021M,p_o55_1021C,p_o55_10P,p_o55_21P
0,Phoenix,338099.0,8533.645001,1.534351,292753.0,7921.511661,1.644905,267029.0,7834.840968,1.783634,...,22098.0,11283.238232,31.039512,0.201971,0.193809,102681.0,12582.401917,7.449164,0.184224,0.227578
1,Maricopa,809639.0,7021.664689,0.527209,762589.0,7050.707979,0.562052,866348.0,11920.184059,0.836421,...,84650.0,9711.135207,6.973923,0.199246,0.188418,371163.0,18927.741519,3.100047,0.226356,0.275211
2,US,61178747.0,68071.820256,0.067640,65914697.0,61977.622534,0.057159,77195489.0,105966.835085,0.083447,...,-3878817.0,98024.267602,1.536273,0.213075,0.186915,21511559.0,168350.595449,0.475748,0.249541,0.297406
3,AZ,1277639.0,9655.846312,0.459427,1249064.0,9375.584302,0.456297,1620055.0,17087.890156,0.641199,...,46856.0,14248.784088,18.486159,0.194748,0.178101,593941.0,26348.646550,2.696802,0.252591,0.304274
4,San Antonio,293401.0,4991.241729,1.034144,255360.0,4913.128433,1.169605,277706.0,6739.814760,1.475356,...,3556.0,8586.343343,146.784635,0.191373,0.178334,63245.0,11287.125719,10.849039,0.208119,0.234837
5,Dallas,310937.0,7631.623418,1.492034,231995.0,6538.375180,1.713268,216916.0,6721.048505,1.883560,...,2260.0,10555.353523,283.921605,0.192880,0.181813,73751.0,11089.819881,9.140936,0.180343,0.225596
6,Jacksonville,182403.0,3648.919155,1.216092,174991.0,3681.648408,1.278971,183392.0,5436.047093,1.801926,...,3517.0,5925.573896,102.421740,0.212544,0.186993,76282.0,9515.290852,7.582878,0.222748,0.272017


In [12]:
Y0Y1.to_excel('output/acs1_age_1021.xlsx',index=False)

#### Urban Villages

In [ ]:
## WILL FILL THIS IN LATER